In [1]:
import cudf as pd
import cupy as cp
import numpy as np
# import matplotlib.pyplot as plt
# import pandas

In [2]:
# Загружаю датасет по депозитам

deposit_data = pd.read_csv('data/deposit_data.csv', index_col=0)

In [3]:
deposit_data = deposit_data.reset_index().drop('index', axis=1)

In [29]:
deposit_data.head(2)

,user_id,campaign_type,source_medium,utm_campaign,interface,campaign_cost,user_phone,id,event_type,event_status,payout,product_type,time_from_reg_till_event,time_from_camp_start_till_event
0,70391e123115e9d0f463495adfa9ff91,show,yandex / organic,client_rebrand_megamaket,app,0.0,79143423444,aa154add98285bd92c7bbe42f602ef7218a45b3d7ce0bf...,show,<NA>,0.0,deposits,152 days 03:39:49.120132000,92 days 16:47:29.443329000
1,70391e123115e9d0f463495adfa9ff91,click,yandex / organic,bk_adv_client,web,0.0,79143423444,416ce92d431422cffd83519bbdddebba9f92718b21e9d9...,click,<NA>,0.0,deposits,152 days 03:39:49.120132000,82 days 19:27:19.982213000


In [50]:
# Сохраняю названия столбцов в виде numpy-массива

cols = np.array(deposit_data.columns)

In [51]:
cols

array(['user_id', 'campaign_type', 'source_medium', 'utm_campaign',
       'interface', 'campaign_cost', 'user_phone', 'id', 'event_type',
       'event_status', 'payout', 'product_type',
       'time_from_reg_till_event', 'time_from_camp_start_till_event'],
      dtype=object)

In [6]:
# Удаляю из массива столбцов ненужные

cols = np.delete(cols, [0,5,6,7,10,11,12,13])

In [ ]:
# Применяю метод value_counts() к каждому столбцу из массива

for col in cols:
    print(col)
    print('-------------------------')
    print(deposit_data[col].value_counts())
    print('::::::::::::::::::::::::::')
    print('\n')

In [8]:
# Создаю подмассив с ID пользователей и разделенными значениями из столбца campaign_type

deposit_data_by_campaign_type = pd.get_dummies(deposit_data[['user_id', 'campaign_type']], columns=['campaign_type'], dtype='int')

In [9]:
# Группирую значения в массиве по ID пользователя. Таким образом в каждом столбце для каждого пользователя суммируется кол-во каждого из трех типов касаний 

deposit_data_by_campaign_type = deposit_data_by_campaign_type.groupby('user_id').sum()

In [10]:
# Выделяю пользователей, у которых была хотя бы одна конверсия

deposit_data_by_campaign_type_conversions = deposit_data_by_campaign_type.loc[deposit_data_by_campaign_type['campaign_type_conversion'] > 0]

In [11]:
# Сортирую этих пользователей по количеству конверсий

deposit_data_by_campaign_type_conversions.sort_values(by='campaign_type_conversion', ascending=False).head(5)

,campaign_type_click,campaign_type_conversion,campaign_type_show
user_id,,,
067c102c89732a569567d2f673dbf0c9,2,15,2
558fa53918641cdaec9c564da667d2f0,1,12,1
8e7e0913bd4ad9c5b0501ad8c8dfe64b,1,12,1
8b436194ccd8675e7c6a0cb8076aeadc,1,12,2
14bd689a3dcbda5e6db725ac450ac3b0,1,12,1


1. Общее количество касаний зарегистрированных пользователей

In [24]:
# Суммы касаний кликов, показов и конверсий

sum_of_touches = deposit_data_by_campaign_type.sum(axis=0)

In [28]:
sum_of_touches

campaign_type_click          124148
campaign_type_conversion       5398
campaign_type_show          1990110
dtype: int64

In [25]:
# Сумма показов без конверсий

sum_of_touches_wo_conversions = sum_of_touches['campaign_type_click'] + sum_of_touches['campaign_type_show']

In [26]:
sum_of_touches_wo_conversions

2114258

In [27]:
deposit_data_by_campaign_type_conversions.sum()

campaign_type_click         1181
campaign_type_conversion    5398
campaign_type_show          1271
dtype: int64

In [30]:
# Сумма касаний, которые привели к конверсиям

sum_of_touches_lead_to_conversions = deposit_data_by_campaign_type_conversions.sum()['campaign_type_click'] + deposit_data_by_campaign_type_conversions.sum()['campaign_type_show']

In [31]:
sum_of_touches_lead_to_conversions

2452

In [36]:
# Процент касаний, который привел к конверсиям:

conversion_touches_percent = round(sum_of_touches_lead_to_conversions / sum_of_touches_wo_conversions * 100, 3)

In [37]:
conversion_touches_percent

0.116

In [39]:
# Общие затраты на рекламу для депозитов

total_expenditures = deposit_data['campaign_cost'].sum()

In [40]:
# Общий доход как сумма столбца payout

total_payout = deposit_data['payout'].sum()

In [46]:
# Общая прибыль от депозитов:

gross_margin = round(total_payout - total_expenditures, 3)

In [47]:
gross_margin

1139472.995

In [48]:
# Процент прибыли

gross_margin_percent = round(gross_margin / total_expenditures * 100, 3)

In [49]:
gross_margin_percent

16.528

In [58]:
deposit_data_dummies_all = pd.get_dummies(deposit_data[['user_id', 'campaign_type', 'source_medium', 
                                                        'utm_campaign', 'interface', 'campaign_cost', 
                                                        'event_type', 'event_status', 
                                                        'payout', 'product_type', 
                                                        'time_from_reg_till_event', 'time_from_camp_start_till_event']], 
                                          columns=['campaign_type', 'source_medium', 
                                                   'interface', 'event_type', 'event_status', 
                                                    'product_type'], 
                                          dtype='int')

In [59]:
deposit_data_dummies_all.head(2)

,user_id,utm_campaign,campaign_cost,payout,time_from_reg_till_event,time_from_camp_start_till_event,campaign_type_click,campaign_type_conversion,campaign_type_show,source_medium_android / mobile_app,...,interface_app,interface_web,event_type_click,event_type_conversion,event_type_show,event_status_approve,event_status_deal,event_status_reject,event_status_sale,product_type_deposits
0,70391e123115e9d0f463495adfa9ff91,client_rebrand_megamaket,0.0,0.0,152 days 03:39:49.120132000,92 days 16:47:29.443329000,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1,70391e123115e9d0f463495adfa9ff91,bk_adv_client,0.0,0.0,152 days 03:39:49.120132000,82 days 19:27:19.982213000,1,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [61]:
deposit_data_dummies_all.columns

Index(['user_id', 'utm_campaign', 'campaign_cost', 'payout',
       'time_from_reg_till_event', 'time_from_camp_start_till_event',
       'campaign_type_click', 'campaign_type_conversion', 'campaign_type_show',
       'source_medium_android / mobile_app', 'source_medium_bing / organic',
       'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
       'source_medium_google / organic', 'source_medium_ios / mobile_app',
       'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
       'source_medium_regular / email', 'source_medium_regular / sms',
       'source_medium_ru.android.bankproduct / mobile_app',
       'source_medium_ru.android.bankproduct_rustore / mobile_app',
       'source_medium_trigger / sms', 'source_medium_vk / cpc',
       'source_medium_web.telegram.org / referral ',
       'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
       'source_medium_yandex / cpc', 'source_medium_yandex / organic',
       'source_medium_zen.yan

In [60]:
# Расчет процента самостоятельных конверсий. Самостоятельные конверсии - те, которые произошли без прямого воздействия рекламы (раньше старта рекламной кампании и нулевой стоимостью рекламной кампании)

deposit_data_dummies_all_conversions = deposit_data_dummies_all.loc[deposit_data_dummies_all['campaign_type_conversion'] > 0]

In [79]:
deposit_data_dummies_all_conversions_self = deposit_data_dummies_all_conversions.loc[(deposit_data_dummies_all_conversions['time_from_camp_start_till_event'].str.contains('-')) & (deposit_data_dummies_all_conversions['campaign_cost'] == 0)]

In [80]:
deposit_data_dummies_all_conversions_self.head()

,user_id,utm_campaign,campaign_cost,payout,time_from_reg_till_event,time_from_camp_start_till_event,campaign_type_click,campaign_type_conversion,campaign_type_show,source_medium_android / mobile_app,...,interface_app,interface_web,event_type_click,event_type_conversion,event_type_show,event_status_approve,event_status_deal,event_status_reject,event_status_sale,product_type_deposits
4993,81cc8fb0573a1b0cf785c2d6ccc7b03c,target_rebrand_megamaket,0.0,0.0000,127 days 18:19:25.694112000,-51 days +23:38:06.101574000,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
4995,81cc8fb0573a1b0cf785c2d6ccc7b03c,megamaket_light_clickarrow,0.0,0.0000,127 days 18:19:25.694112000,-50 days +12:29:13.263522000,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
15292,b0a89696c3848c9b7487f55e00222214,rebrand_target_response,0.0,276.6546,40 days 15:23:52.035882000,-30 days +22:19:19.330277000,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1
15293,b0a89696c3848c9b7487f55e00222214,rebrand_target_response,0.0,0.0000,40 days 15:23:52.035882000,-30 days +22:19:19.330277000,0,1,0,0,...,1,0,0,1,0,0,0,0,1,1
15294,b0a89696c3848c9b7487f55e00222214,rebrand_target_response,0.0,0.0000,40 days 15:23:52.035882000,-30 days +22:19:19.330277000,0,1,0,0,...,1,0,0,1,0,1,0,0,0,1


In [85]:
self_conversions = round(deposit_data_dummies_all_conversions_self['campaign_type_conversion'].sum() / deposit_data_dummies_all['campaign_type_conversion'].sum() * 100, 3)

In [86]:
self_conversions

33.883

In [89]:
# Процент одобрений банком - процент одобренных депозитов от общего числа конверсий

bank_positive_decisions = deposit_data_dummies_all_conversions.loc[(deposit_data_dummies_all_conversions['event_status_approve'] != 0) | (deposit_data_dummies_all_conversions['event_status_deal'] != 0)][['event_status_approve', 'event_status_deal']].sum()

In [91]:
bank_positive_decisions_sum = bank_positive_decisions['event_status_approve'] + bank_positive_decisions['event_status_deal']

In [96]:
bank_positive_decisions_percent = round(bank_positive_decisions_sum / deposit_data_dummies_all['campaign_type_conversion'].sum() * 100, 3)

In [97]:
bank_positive_decisions_percent

72.268